## Рекомендательные системы

В этом ноутбуке мы применим алгоритм коллаборативной фильтрации на item-base подходе. Работать мы будем с датасетом MovieLens, который содержит в себе информацию об оценках фильмов пользователями одноименного сайта.

Давайте загрузим необходимые библиотеки.

In [0]:
import zipfile
from collections import defaultdict, Counter
import datetime

from scipy import linalg
import scipy.sparse as sps
import numpy as np
import matplotlib.pyplot as plt

Скачаем данные

In [0]:
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

--2020-04-11 16:21:20--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  18.2MB/s    in 0.3s    

2020-04-11 16:21:21 (18.2 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]



Распакуем данные и посмотрим, как они устроены.

In [0]:
with zipfile.ZipFile("ml-1m.zip", "r") as z:
    print("files in archive")
    print(z.namelist())
    print("movies")
    with z.open("ml-1m/movies.dat") as m:
        print(str(m.readline()).split("::"))
    print("users")
    with z.open("ml-1m/users.dat") as m:
        print(str(m.readline()).split("::"))
    print("ratings")
    with z.open("ml-1m/ratings.dat") as m:
        print(str(m.readline()).split("::"))

files in archive
['ml-1m/', 'ml-1m/movies.dat', 'ml-1m/ratings.dat', 'ml-1m/README', 'ml-1m/users.dat']
movies
['b"1', 'Toy Story (1995)', 'Animation|Children\'s|Comedy\\n"']
users
["b'1", 'F', '1', '10', "48067\\n'"]
ratings
["b'1", '1193', '5', "978300760\\n'"]


Мы видим, что в архиве лежит информация о фильмах. Это movieId фильма, название и жанр. О пользователях нам известен userId, пол (F, M), возраст, закодированная информация о трудоуствройстве и zip-code. И информация о рейтинге: userId, movieId, оценка и момент времени, когда оценка была сделана. Давайте прочитаем данные.

In [0]:
# read data
movies = {} # id
users = {} # id
ratings = defaultdict(list) # user-id

with zipfile.ZipFile("ml-1m.zip", "r") as z:
    # parse movies
    with z.open("ml-1m/movies.dat") as m:
        for line in m:
            MovieID, Title, Genres = line.decode('iso-8859-1').strip().split("::")
            MovieID = int(MovieID)
            Genres = Genres.split("|")
            movies[MovieID] = {"Title": Title, "Genres": Genres}
    
    # parse users
    with z.open("ml-1m/users.dat") as m:
        fields = ["UserID", "Gender", "Age", "Occupation", "Zip-code"]
        for line in m:
            row = list(zip(fields, line.decode('iso-8859-1').strip().split("::")))
            data = dict(row[1:])
            data["Occupation"] = int(data["Occupation"])
            users[int(row[0][1])] = data
    
    # parse ratings
    with z.open("ml-1m/ratings.dat") as m:
        for line in m:
            UserID, MovieID, Rating, Timestamp = line.decode('iso-8859-1').strip().split("::")
            UserID = int(UserID)
            MovieID = int(MovieID)
            Rating = int(Rating)
            Timestamp = int(Timestamp)
            ratings[UserID].append((MovieID, Rating, datetime.datetime.fromtimestamp(Timestamp)))

Посмотрим на данные

In [0]:
print(users[3])
print(ratings[3])

{'Gender': 'M', 'Age': '25', 'Occupation': 15, 'Zip-code': '55117'}
[(3421, 4, datetime.datetime(2000, 12, 31, 21, 29, 7)), (1641, 2, datetime.datetime(2000, 12, 31, 21, 33, 50)), (648, 3, datetime.datetime(2000, 12, 31, 21, 24, 27)), (1394, 4, datetime.datetime(2000, 12, 31, 21, 29, 7)), (3534, 3, datetime.datetime(2000, 12, 31, 21, 11, 8)), (104, 4, datetime.datetime(2000, 12, 31, 21, 34, 46)), (2735, 4, datetime.datetime(2000, 12, 31, 21, 24, 27)), (1210, 4, datetime.datetime(2000, 12, 31, 21, 20)), (1431, 3, datetime.datetime(2000, 12, 31, 21, 11, 35)), (3868, 3, datetime.datetime(2000, 12, 31, 21, 34, 46)), (1079, 5, datetime.datetime(2000, 12, 31, 21, 31, 36)), (2997, 3, datetime.datetime(2000, 12, 31, 21, 29, 7)), (1615, 5, datetime.datetime(2000, 12, 31, 21, 21, 50)), (1291, 4, datetime.datetime(2000, 12, 31, 21, 20)), (1259, 5, datetime.datetime(2000, 12, 31, 21, 31, 36)), (653, 4, datetime.datetime(2000, 12, 31, 21, 22, 37)), (2167, 5, datetime.datetime(2000, 12, 31, 21, 20))

Теперь разобьем данные на тренировочную и тестовую выборку. Разбивать будем рейтинги по времени. Для начала найдем дату на которую было выставленно 80% рейтингов в датасете. И все рейтинги, что были до, попадут в train, а остальные в test.

In [0]:
times = []
for user_ratings in ratings.values():
  times.extend([x[2] for x in user_ratings])

In [0]:
times = sorted(times)

In [0]:
threshold_time = times[int(0.8 * len(times))]

In [0]:
train = []
test = []
for user_id, user_ratings in ratings.items():
    train.extend((user_id, rating[0], rating[1] / 5.0) for rating in user_ratings if rating[2] <= threshold_time)
    test.extend((user_id, rating[0], rating[1] / 5.0) for rating in user_ratings if rating[2] > threshold_time)
print("ratings in train:", len(train))
print("ratings in test:", len(test))

ratings in train: 800168
ratings in test: 200041


Сейчас мы хотим посчититать похожесть по фильмам. Для этого нам сначала нужно посчитать средний рейтинг каждого пользователя.

In [0]:
user_average = defaultdict(list)
for u, i, r in train:
    user_average[u].append(r)
for u in user_average.keys():
    user_average[u] = sum(user_average[u]) / len(user_average[u])
user_average = dict(user_average)

Вычтем среднее и посчитаем норму по каждому item'y

In [0]:
item_norms = defaultdict(float)
for u, i, r in train:
    item_norms[i] += (r - user_average[u]) ** 2
for i in item_norms.keys():
    item_norms[i] = item_norms[i] ** 0.5
item_norms = dict(item_norms)

Посчитаем обновленную матрицу рейтингов, где из каждого рейтинга вычтем среднюю оценку по пользователю и поделим на норму вектора фильма.

In [0]:
item_user = sps.csc_matrix(
    ([(r - user_average[u]) / (item_norms[i] + 1e-6) for u, i, r in train],
    ([e[1] for e in train], [e[0] for e in train]))
)

В конечном итоге, мы можем посчитать матрицу похожести фильмов через скалярное произведение.

In [0]:
item_item_similarity = item_user.dot(item_user.transpose())

Давайте обнулим те похожести, которые меньше 0. Это означает, что если угол между векторами больше 90 градусов, то мы считаем их совсем не похожими.

In [0]:
item_item_similarity[item_item_similarity < 0] = 0

In [0]:
train_by_user = defaultdict(list)
test_by_user = defaultdict(list)
for u, i, r in train:
    train_by_user[u].append((i, r))
for u, i, r in test:
    test_by_user[u].append((i, r))

Давайте построим рекомендацию для некоторого пользователя. Например с id=6040. Вытащим из обучающей части его рейтинги.

In [0]:
user_id = 6040
user_ratings = train_by_user[user_id]

user_rated_items = [e[0] for e in user_ratings]
user_rated_ratings = sps.csr_matrix([[e[1]] for e in user_ratings])

Найдем столбцы в матрице похожести, которые соответствуют оцененным фильмам.

In [0]:
similar_items = item_item_similarity[:, user_rated_items]

Теперь мы смешаем пользовательские рейтинги с весами похожести фильмов и таким образом получим предсказанные рейтинги.

In [0]:
recoms = similar_items.dot(user_rated_ratings) / (similar_items.sum(axis=1) + 1e-6)

In [0]:
recoms

matrix([[0.        ],
        [0.74819368],
        [0.40285119],
        ...,
        [0.79236067],
        [0.76070987],
        [0.75454628]])

Можем посмотреть на рейтинги в тесте этого пользователя и сравнить с предсказанными.

In [0]:
test_user_ratings = test_by_user[user_id]

In [0]:
test_user_rated_items = [e[0] for e in test_user_ratings]
test_user_rated_ratings = np.array([[e[1]] for e in test_user_ratings])

In [0]:
np.array(recoms[test_user_rated_items, :])[:, 0]

array([0.81907084, 0.77952099, 0.7431567 , 0.78205121, 0.8146596 ,
       0.75421669, 0.82053979, 0.79268059, 0.77863912, 0.78497366,
       0.80319477, 0.82919131, 0.79921779, 0.77875818, 0.72230169,
       0.78808294, 0.75104579, 0.76944995, 0.78971198, 0.79084428,
       0.71691417, 0.75438915, 0.74879743])

In [0]:
test_user_rated_ratings[:, 0]

array([0.8, 1. , 0.6, 1. , 1. , 0.8, 0.8, 0.8, 0.8, 0.8, 1. , 1. , 0.8,
       0.8, 0.6, 0.8, 0.8, 0.8, 0.8, 0.8, 0.6, 0.8, 0.6])

Хорошо видно, что для тех фильмов, где рейтинг 0.6, предсказанные значения тоже наиболее низкие.

Посчитаем метрику.

In [0]:
mse = 0
COUNT = 0
for user, ratings in train_by_user.items():
    user_rated_items = [e[0] for e in ratings]
    user_rated_ratings = sps.csr_matrix([[e[1]] for e in ratings])
    similar_items = item_item_similarity[:, user_rated_items]
    recoms = similar_items.dot(user_rated_ratings) / (similar_items.sum(axis=1) + 1e-6)
    tbu = test_by_user[user]
    test_items = [e[0] for e in tbu]
    test_ratings = np.array([[e[1]] for e in tbu])
    errors = np.square(recoms[test_items, :] - test_ratings)
    mse += sum(errors)
    COUNT += len(errors)
print(mse / COUNT)

[[0.03749982]]


## ALS факторизация

В этом подходе оценка $r_{ui}$ пользователя $u$, поставленная фильму $i$, ищется как скалярное произведение векторов $p_u$ и $q_i$ в некотором пространстве $\mathbb{R}^K$ латентных признаков:

$$
	\hat{r}_{ui} = p_u^T q_i 
$$


Иными словами, модель находит пространство признаков, в котором мы описываем и фильмы и пользователей и в котором рейтинг является мерой близости между фильмами и пользователями.
	
Для настройки модели будем минимизировать следующую ошибку:
	$$
	\sum_{(u, i, r_{ui})} (r_{ui} - p_u^T q_i)^2 + \lambda_{p} p_u^T p_u + \lambda_{q} q_i^T q_i,
	$$
	где суммирование ведется по всем тройкам $(u, i, r_{ui})$ выборки, слагаемые с $\lambda_{p}$ и $\lambda_{q}$ добавлены для регуляризации.


In [0]:
LATENT_SIZE = 10

# регуляризаторы
lambda_p = 0.2
lambda_q = 0.001



Рассмотрим работу алгоритма ALS, продолжая работать с данными Movielens.

Посчитаем количество пользователей и фильмов

In [0]:
n_users = max([e[0] for e in train]) + 1
n_items = max([e[1] for e in train]) + 1

Инициализируем латентные представления для пользователей

In [0]:
p = 0.1 * np.random.random((n_users, LATENT_SIZE))
q = 0.1 * np.random.random((n_items, LATENT_SIZE))

Составим словарь взаимодействий по фильму

In [0]:
train_by_item = defaultdict(list)
for u, i, r in train:
    train_by_item[i].append((u, r))

Теперь составим матрицу $P$ из векторов $p_u$ и матрицу $Q$ из векторов $q_i$. Матрицей $Q[u] \in \mathbb{R}^{n_u \times K}$ будем обозначать подматрицу матрицы $Q$ только для товаров, оцененных пользователем $u$, где $n_u$ - количество оценок пользователя $u$.
	
Шаг перенастройки $p_u$ при фиксированной матрице $Q$ сводится к настройке ridge-регрессии и выглядит так:
	$$
	A_u = Q[u]^T Q[u] \\
	d_u = Q[u]^T r_u \\
	p_u = (\lambda_p n_u I + A_u)^{-1} d_u
	$$

In [0]:
def compute_p(p, q, train_by_user):
    for u, rated in train_by_user.items():
        rated_items = [i for i, _ in rated]
        rated_scores = np.array([r for _, r in rated])
        Q = q[rated_items, :]
        A = (Q.T).dot(Q)
        d = (Q.T).dot(rated_scores)
        p[u, :] = np.linalg.solve(lambda_p * len(rated_items) * np.eye(LATENT_SIZE) + A, d)
    return p

p = compute_p(p, q, train_by_user)


In [0]:
p.shape

(6041, 10)


Формулы для обновления $q_i$ при фиксированной матрице $P$ выглядят аналогично, реализация будет выглядеть следующим образом:


In [0]:
def compute_q(p, q, train_by_item):
    for i, rated in train_by_item.items():
        rated_users = [j for j, _ in rated]
        rated_scores = np.array([s for _, s in rated])
        P = p[rated_users, :]
        A = (P.T).dot(P)
        d = (P.T).dot(rated_scores)
        q[i, :] = np.linalg.solve(lambda_q * len(rated_users) * np.eye(LATENT_SIZE) + A, d)
    return q

q = compute_q(p, q, train_by_item)

In [0]:
q.shape

(3953, 10)

Теперь мы можем сделать предсказание всей матрицы оценок

In [0]:
predictions = p.dot(q.T)

Качество работы модели будем определять по среднеквадратичной ошибке между рейтингом и предсказанием

In [0]:
def train_error(predictions):
    return np.mean([(predictions[u, i] - r) ** 2 for u, i, r in train])

def test_error(predictions):
    return np.mean([(predictions[u, i] - r) ** 2 for u, i, r in test])

Теперь полностью реализуем метод: в ALS проводятся $N$ итераций, в рамках каждой сначала оптимизируется $p$ при фиксированном $q$, затем $q$ при фиксированном $p$.

In [0]:
%%time
N_ITER = 20
for iter in range(N_ITER):
    p = compute_p(p, q, train_by_user)
    q = compute_q(p, q, train_by_item)

    predictions = p.dot(q.T)
    
    print(iter, train_error(predictions), test_error(predictions))

0 0.03066430994776746 0.14848666531492907
1 0.027029643468250878 0.14074191138275235
2 0.025794728422240832 0.1339690162458141
3 0.025298580265259298 0.1278527884340578
4 0.02506238514034076 0.1223956056524342
5 0.02492678880598967 0.11752415102621717
6 0.024839114664988483 0.1131554138158145
7 0.02477830320063082 0.10922270488662998
8 0.024734253691901246 0.10567243208608301
9 0.02470142627897532 0.10246051641096007
10 0.02467650673841401 0.09954996820782655
11 0.024657359817865286 0.09690909301062624
12 0.024642533578697334 0.09451025370889021
13 0.024631017307410807 0.09232905873704036
14 0.0246221022146939 0.09034386681955267
15 0.024615274025963453 0.08853545832857632
16 0.02461011695772306 0.08688675324959096
17 0.024606257068772123 0.08538255403089305
18 0.02460335391426012 0.08400930982939459
19 0.024601109696852287 0.08275490265854599
CPU times: user 1min 5s, sys: 1min 17s, total: 2min 22s
Wall time: 49.6 s
